In [ ]:
# API Keys


In [94]:
import re
from typing import List, Dict, Tuple
from huggingface_hub import InferenceClient
from transformers import pipeline
from datasets import load_dataset

client = InferenceClient(
    api_key=HUGGING_FACE_TOKEN,
    provider="auto"   # Automatically selects best provider
)

emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)


Device set to use cuda:0
/home/cyra/code/MindPal_Backend/AI/venv/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# from google import genai

# client = genai.Client(api_key=GOOGLE_API_KEY)

# response = client.models.generate_content(
#     model="gemini-2.5-pro", contents=prompt
# )
# print(response.text)

In [92]:
user_input = "My parents found out I'm gay, but they don't accept it, I want to leave this place, as far as possible, I don't want to see them again"

In [ ]:
def emotion_detection(user_input: str) -> str:
    emotion = emotion_classifier(user_input)
    return emotion[0]

detected_emotion = emotion_detection(user_input)
print(detected_emotion)

[{'label': 'anger', 'score': 0.008162119425833225}, {'label': 'disgust', 'score': 0.0042908210307359695}, {'label': 'fear', 'score': 0.006158373784273863}, {'label': 'joy', 'score': 0.003666603472083807}, {'label': 'neutral', 'score': 0.005691845901310444}, {'label': 'sadness', 'score': 0.969331681728363}, {'label': 'surprise', 'score': 0.002698499010875821}]


In [ ]:
def normalize_text(text: str) -> str:
    t = text.strip()
    # basic normalization
    t = re.sub(r"\s+", " ", t)
    return t

# Load slang dataset into a mapping: slang -> canonical meaning(s) (examples)
def load_genz_slang_map() -> Dict[str, List[str]]:
    """
    Try to fetch a structured GenZ slang mapping from available datasets.
    """
    try:
        # Attempt to load MLBtrio genz-slang-dataset using datasets or Hugging Face
        ds = load_dataset("MLBtrio/genz-slang-dataset", split="train")
        slang_map = {}
        # dataset may have 'slang' and 'meaning' fields (check variety)
        for row in ds:
            key = row.get("Slang")
            desc = row.get("Description")

            if key:
                slang_map[key.lower()] = [desc] if desc else []
        if slang_map:
            return slang_map
    except Exception as e:
        print(f"Failed to load GenZ slang dataset: {e}")
        pass

SLANG_MAP = load_genz_slang_map()

In [112]:
def detect_slang_candidates(text: str) -> List[Tuple[str, List[str]]]:
    """
    Find slang tokens present in text and return candidate meanings.
    """
    lower = text.lower()
    found = []
    for s in SLANG_MAP:
        # match whole word or phrase
        if re.search(r"\b" + re.escape(s) + r"\b", lower):
            found.append((s, SLANG_MAP[s]))
    return found

text = normalize_text("Got the job today, big W!")
print(text)

sl = detect_slang_candidates(text)
print(sl)


Got the job today, big W!
[('w', ['Shorthand for win'])]


In [109]:
def map_slang_in_text(text: str) -> Tuple[str, List[str]]:
    """
    Returns an augmented text with slang replaced by canonical meaning (in brackets),
    and a list of detected mapped meanings for use by the emotion detector.
    """
    context = text
    candidates = detect_slang_candidates(text)
    slang_token, meanings = candidates[0][0], candidates[0][1]

    # replace slang with "slang (meaning)" to give classifier better signal
    context = re.sub(r"\b" + re.escape(slang_token) + r"\b", f"{slang_token} ({meanings[0]})", context, flags=re.IGNORECASE)
    return context

rep = map_slang_in_text("Michael Jordan is the GOAT of basketball.")
print(rep)

Michael Jordan is the goat (Greatest of all Times) of basketball.


In [ ]:
prompt = f"""
You are MindPal, a supportive wellbeing chatbot for 13-15 year-old australian teens. 
Your responses should be:
- Warm, understanding, and age-appropriate
- Validate their feelings without being condescending
- Use language that feels natural to teens
- Acknowledge and reflect their feeling(s)
- Keep replies within 1-3 sentences and sound like a natural conversation
- Encourage them to talk more, ask follow up questions and let user express their feelings
- Encourage real-life support systems and resources
- When appropriate and you have enough information, gently encourage the teen to talk with a trusted adult or friend
- Avoid shaming or lecturing
- Do NOT encourage any dangerous behaviour or provide inappropriate information
- Do NOT give medical or clinical advice or replace professional help
- Do NOT overly sugarcoat things, be natural and honest when necessary
- Do NOT let the user give out any personal information
- When the user ask general questions that are unrelated to your purpose, politely decline to answer and redirect the conversation back

Current emotion(s) detected: {detected_emotion}

Always rethink and double check your answer before responding.
When you completely understand you can start the session.
"""

messages = [
    {"role": "system", "content": prompt},
    {"role": "user", "content": user_input}
]

# Chat completion
completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1",
    messages=messages,
    temperature=0.3,
    top_p=0.9
)

raw_reply = completion.choices[0].message.content

print(raw_reply)

<think>
We are given that the user is expressing a desire to leave home due to parental rejection after coming out as gay. The emotion detection shows very high sadness (0.97) and low other emotions. 

Key points to address:
- The user is in a 13-15 age group (Australian teen)
- They are feeling rejected and hurt by their parents' lack of acceptance
- They express a strong desire to run away

Our response must:
1. Validate their feelings: Acknowledge the pain and difficulty of the situation without judgment.
2. Avoid encouraging dangerous behavior (like running away) but without dismissing their feelings.
3. Gently encourage seeking real-life support.
4. Keep the response warm, natural, and within 1-3 sentences.

Approach:
- Start by acknowledging their pain: "That sounds incredibly tough and hurtful."
- Validate their feelings: "It's completely understandable to feel upset and want to leave when the people who are supposed to support you aren't accepting."
- Gently discourage impulsiv

In [89]:
reply = raw_reply.split("</think>")[-1].strip()

print(reply)

That sounds incredibly tough and painful to feel so rejected by your parents. It's completely understandable to want to escape that hurt right now. Have you got anyone safe you can talk to about this, like a trusted teacher or friend who gets you?  

Running away can put you in really unsafe situations - maybe we could talk about ways to feel safer where you are first?


In [ ]:
TEEN_SUPPORT_PROMPT = """

Current emotion detected: {emotion}
Conversation context: {context}

Teen's message: "{message}"

Respond as a supportive friend who truly understands what they're going through.
"""

COPING_STRATEGY_PROMPT = """
Based on the teen's emotional state ({emotion}) and situation ({context}), 
recommend 2-3 age-appropriate coping strategies from our database:

{coping_strategies_db}

Format as friendly suggestions, not instructions. Include relevant resource links.
"""

PARENT_TIPS_PROMPT = """
A parent needs guidance on how to support their teenager who is experiencing {emotion}.

Recent conversation themes: {themes}
Recommended activities teen is doing: {activities}

Provide 3-4 specific, actionable tips for the parent to:
1. Approach conversations with empathy
2. Create supportive environment
3. Know when to seek professional help
4. Respect teen's autonomy while staying involved

Keep advice practical and evidence-based.
"""

'- When appropriate and you have enough information, recommend 1-2 coping strategies, or gently encourage the teen to talk with a trusted adult or friend'